In [1]:
import lsst.daf.butler as dafButler
import numpy as np


## From Shany:

In [2]:

output_collection = "DECam/runs/merian/dr1_wide"
data_type = "deepCoadd_calexp"

skymap = "hsc_rings_v1"

butler = dafButler.Butler('/projects/MERIAN/repo/', collections=output_collection, skymap=skymap)


In [3]:

patches = np.array([[data_id['tract'], data_id["patch"]] for data_id in butler.registry.queryDataIds (['tract','patch'], datasets=data_type, 
                                                 collections=output_collection, skymap=skymap)])
patches = patches[patches[:, 0].argsort()]

In [4]:
tracts, idx = np.unique(patches[:,0], return_index=True) 
patches_by_tract = np.split(patches[:,1] ,idx[1:])

In [5]:
# f = open("reducedPatches.csv", "w")
# for i, tract in enumerate(tracts):
#     f.write(f'{tract}')

#     patches_i = np.unique(patches_by_tract[i])
#     patches_i.sort()
#     for p in patches_i:
#         f.write(f',{p}')
#     f.write("\n")

# f.close()


---

## Used in `gaap`:

In [6]:
merian_patches = [item.dataId for item in butler.registry.queryDatasets('objectTable', 
                        dataId=dict(tract=9813), collections=output_collection,
                        skymap=skymap)]

In [7]:
np.unique(merian_patches)

array([GovernorDimension(skymap), DatabaseDimension(tract),
       DatabaseDimension(patch)], dtype=object)

In [8]:
butler = dafButler.Butler('/projects/MERIAN/repo/')

[item.dataId for item in butler.registry.queryDatasets('objectTable', 
                        dataId=dict(tract=9813, patch=3), collections='DECam/runs/merian/dr1_wide',
                        skymap='hsc_rings_v1')]

[]

In [9]:
IDs = butler.registry.queryDataIds(['tract','patch'], datasets=data_type, 
                                            collections=output_collection, skymap=skymap)

In [10]:
for id in butler.registry.queryDataIds(['tract','patch'], datasets=data_type, 
                                            collections=output_collection, skymap=skymap):
    if (id["tract"] == 9813) & (id["patch"] == 3):
        print(id)

{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}
{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}
{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}
{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}
{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}
{skymap: 'hsc_rings_v1', tract: 9813, patch: 3}


In [11]:
for data_id in butler.registry.queryDataIds(['tract','patch'], datasets=data_type, 
                                            collections=output_collection, skymap=skymap):
    print(data_id)

{skymap: 'hsc_rings_v1', tract: 9570, patch: 80}
{skymap: 'hsc_rings_v1', tract: 9572, patch: 49}
{skymap: 'hsc_rings_v1', tract: 9589, patch: 45}
{skymap: 'hsc_rings_v1', tract: 9566, patch: 24}
{skymap: 'hsc_rings_v1', tract: 9593, patch: 33}
{skymap: 'hsc_rings_v1', tract: 9568, patch: 59}
{skymap: 'hsc_rings_v1', tract: 9569, patch: 51}
{skymap: 'hsc_rings_v1', tract: 9574, patch: 77}
{skymap: 'hsc_rings_v1', tract: 9587, patch: 58}
{skymap: 'hsc_rings_v1', tract: 9567, patch: 12}
{skymap: 'hsc_rings_v1', tract: 9593, patch: 62}
{skymap: 'hsc_rings_v1', tract: 9574, patch: 0}
{skymap: 'hsc_rings_v1', tract: 9578, patch: 2}
{skymap: 'hsc_rings_v1', tract: 9578, patch: 59}
{skymap: 'hsc_rings_v1', tract: 9570, patch: 16}
{skymap: 'hsc_rings_v1', tract: 9574, patch: 43}
{skymap: 'hsc_rings_v1', tract: 9589, patch: 51}
{skymap: 'hsc_rings_v1', tract: 9589, patch: 41}
{skymap: 'hsc_rings_v1', tract: 9571, patch: 17}
{skymap: 'hsc_rings_v1', tract: 9595, patch: 38}
{skymap: 'hsc_rings_v1

In [13]:
tract=8282

In [14]:
output_collection = "DECam/runs/merian/dr1_wide"
data_type = "deepCoadd_calexp"
skymap = "hsc_rings_v1"

butler = dafButler.Butler('/projects/MERIAN/repo/', collections=output_collection, skymap=skymap)

patches = np.array([[data_id['tract'], data_id["patch"]] for data_id in butler.registry.queryDataIds (['tract','patch'], datasets=data_type, 
                                                collections=output_collection, skymap=skymap)])
patches = patches[patches[:, 0].argsort()]
del butler 

patches = np.unique(patches[patches[:,0] == tract][:,1])


In [15]:
output_collection = "DECam/runs/merian/dr1_wide"
data_type = "deepCoadd_calexp"
skymap = "hsc_rings_v1"

butler = dafButler.Butler('/projects/MERIAN/repo/', collections=output_collection, skymap=skymap)

patches = np.array([[data_id['tract'], data_id["patch"]] for data_id in butler.registry.queryDataIds (['tract','patch'], datasets=data_type, 
                                                collections=output_collection, skymap=skymap)])
patches = patches[patches[:, 0].argsort()]
# del butler 

patches = np.unique(patches[patches[:,0] == tract][:,1])

----
There is a problem where not every patch returned by the codes above have measurements in N708. Specifically, they don't have `deepCoadd_meas` datasets with data ID (for example): `{band:'N708', skymap:'hsc_rings_v1', tract:8279, patch:54}`.

So if I want to get a list of only patches that have N708, use this code:

----
# Abby's solution:

Okay actually, I need every patch to have the following four data products:

- deepCoadd_ref
- deepCoadd_meas
- deepCoadd_scarletModelData
- deepCoadd_calexp

So I don't actually need an `objecTable`. Let's see if this works.

In [17]:
def findReducedPatches(tract, band="N708", output_collection='DECam/runs/merian/dr1_wide', skymap='hsc_rings_v1'):
        butler = dafButler.Butler('/projects/MERIAN/repo/')

        dataId = dict(tract=tract, band=band, skymap=skymap)
        
        deepCoadd_ref_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_ref', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_meas_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_meas', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_scarletModelData_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_scarletModelData', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_calexp_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_calexp', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        del butler

        patches = deepCoadd_ref_patches.intersection(deepCoadd_meas_patches, deepCoadd_scarletModelData_patches, deepCoadd_calexp_patches)
        return(np.array(list(patches)))

In [192]:
# f = open("../../../reducedPatches_N708.csv", "w")

# for tract in tracts:
#     patches = ",".join(findReducedPatches(tract).astype(str))
#     if len(patches) > 0:
#         f.write(f'{tract},{patches}\n')
# f.close()



In [196]:
# f = open("../../../reducedTracts_n708.csv", "w")
# f.write("Tract,patches\n")
# for tract in tracts:
#     patches = findReducedPatches(tract)
#     if len(patches) > 0:
#         f.write(f'{tract},{len(patches)}\n')
# f.close()

---

## Investigating for Erin

In [18]:
erintracts= [9378,9379,9454,9455,9456,9457,9458,9459,9460,9461,9462,9463,9464,9465,9466,9467,9468,9469,9470,9471,9491,9492,9493,9494,9495,9496,9556,9557,9558,9559,9560,9561,9562,9563,9564,9565]

In [19]:
band = "N540"
for tract in erintracts:
        dataId = dict(tract=tract, band=band, skymap=skymap)

        deepCoadd_ref_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_ref', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_meas_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_meas', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_scarletModelData_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_scarletModelData', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])

        deepCoadd_calexp_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_calexp', 
                dataId=dataId, collections=output_collection,
                skymap=skymap)])


        patches_int = deepCoadd_ref_patches.intersection(deepCoadd_meas_patches, deepCoadd_scarletModelData_patches, deepCoadd_calexp_patches)
        patches_uni = deepCoadd_ref_patches.union(deepCoadd_meas_patches, deepCoadd_scarletModelData_patches, deepCoadd_calexp_patches)
        if len(patches_int) != len(patches_uni):
                print(f"Tract: {tract}")
                print (f"Band: {band}")
                print(f'ref missing: {patches_uni - deepCoadd_ref_patches}')
                print(f'meas missing: {patches_uni - deepCoadd_meas_patches}')
                print(f'scarlet missing: {patches_uni - deepCoadd_scarletModelData_patches}')
                print(f'calexp missing: {patches_uni - deepCoadd_calexp_patches}')
                print()

Tract: 9378
Band: N540
ref missing: {59}
meas missing: set()
scarlet missing: set()
calexp missing: set()

Tract: 9455
Band: N540
ref missing: set()
meas missing: {72, 73, 74, 75, 76, 77, 78, 79}
scarlet missing: set()
calexp missing: {72, 73, 74, 75, 76, 77, 78, 79}

Tract: 9456
Band: N540
ref missing: set()
meas missing: {80, 79}
scarlet missing: set()
calexp missing: {80, 79}

Tract: 9556
Band: N540
ref missing: set()
meas missing: {74}
scarlet missing: set()
calexp missing: {74}

Tract: 9557
Band: N540
ref missing: set()
meas missing: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80}
scarlet missing: set()
calexp missing: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27

In [20]:
skymap_map = butler.get("skyMap", collections="skymaps", skymap="hsc_rings_v1")

def getPatchCenter(skymap, tract, patch):
    """Find the center of the patch BBox in pixel coordinates and convert using tract WCS."""
    wcs = skymap_map.generateTract(tract).getWcs()
    bbox = skymap_map.generateTract(tract).getPatchInfo(patch).getInnerBBox()
    sky = wcs.pixelToSky(
        np.mean((bbox.getBeginX(), bbox.getEndX())),
        np.mean((bbox.getBeginY(), bbox.getEndY()))
    )
    return sky[0].asDegrees(), sky[1].asDegrees()


In [21]:
tract= 9461
print (f"Tract: {tract}")

for patch in range(73,80): 
    print (f"Patch: {patch}")
    ra, dec = getPatchCenter(skymap_map, tract, patch)
    print ('%.8f deg, %.8f deg' %(ra, dec))


Tract: 9461
Patch: 73
348.70803611 deg, 0.00286927 deg
Patch: 74
348.52139784 deg, 0.00286935 deg
Patch: 75
348.33475164 deg, 0.00286939 deg
Patch: 76
348.14810149 deg, 0.00286941 deg
Patch: 77
347.96145133 deg, 0.00286939 deg
Patch: 78
347.77480514 deg, 0.00286935 deg
Patch: 79
347.58816687 deg, 0.00286927 deg


In [22]:
tract= 9461

In [23]:
dataId = dict(tract=tract, band="N708", skymap=skymap)

deepCoadd_ref_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_ref', 
        dataId=dataId, collections=output_collection,
        skymap=skymap)])

deepCoadd_meas_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_meas', 
        dataId=dataId, collections=output_collection,
        skymap=skymap)])

deepCoadd_scarletModelData_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_scarletModelData', 
        dataId=dataId, collections=output_collection,
        skymap=skymap)])

deepCoadd_calexp_patches = set([item.dataId["patch"] for item in butler.registry.queryDatasets('deepCoadd_calexp', 
        dataId=dataId, collections=output_collection,
        skymap=skymap)])

In [24]:
np.array(list(deepCoadd_ref_patches)), np.array(list(deepCoadd_meas_patches)), np.array(list(deepCoadd_scarletModelData_patches)), np.array(list(deepCoadd_calexp_patches))

(array([73, 74, 75, 76, 77, 78, 79]),
 array([], dtype=float64),
 array([73, 74, 75, 76, 77, 78, 79]),
 array([], dtype=float64))

---

# Want to find patches near and around Cosmos

In [25]:
skymap_map = butler.get("skyMap", collections="skymaps", skymap="hsc_rings_v1")

def getPatchCenter(skymap, tract, patch):
    """Find the center of the patch BBox in pixel coordinates and convert using tract WCS."""
    wcs = skymap_map.generateTract(tract).getWcs()
    bbox = skymap_map.generateTract(tract).getPatchInfo(patch).getInnerBBox()
    sky = wcs.pixelToSky(
        np.mean((bbox.getBeginX(), bbox.getEndX())),
        np.mean((bbox.getBeginY(), bbox.getEndY()))
    )
    return sky[0].asDegrees(), sky[1].asDegrees()


In [27]:
getPatchCenter(skymap_map, 9813, 50)

(150.0610557967277, 2.4181047840498073)

In [26]:
for tract in tracts:
    print(tract)

8279
8280
8281
8282
8283
8284
8285
8286
8520
8521
8522
8523
8524
8525
8526
8527
8762
8763
8764
8765
8766
8767
8768
8769
9004
9005
9006
9007
9008
9009
9010
9011
9071
9072
9073
9074
9075
9076
9077
9078
9079
9080
9081
9082
9083
9084
9085
9086
9087
9088
9089
9090
9091
9092
9096
9097
9098
9099
9100
9101
9102
9103
9104
9105
9106
9107
9108
9109
9110
9111
9112
9113
9125
9126
9128
9129
9130
9131
9132
9133
9134
9135
9136
9226
9227
9247
9248
9249
9250
9251
9252
9253
9313
9314
9315
9316
9317
9318
9319
9320
9321
9322
9323
9324
9325
9326
9327
9328
9329
9330
9331
9332
9333
9334
9335
9336
9339
9340
9341
9342
9343
9344
9345
9346
9347
9348
9349
9350
9351
9352
9353
9354
9355
9356
9367
9368
9369
9371
9372
9373
9374
9375
9376
9377
9378
9379
9455
9456
9457
9458
9459
9461
9462
9463
9464
9465
9466
9467
9468
9469
9470
9471
9492
9493
9494
9495
9496
9556
9557
9558
9559
9560
9561
9562
9563
9564
9565
9566
9567
9568
9569
9570
9571
9572
9573
9574
9575
9576
9577
9578
9579
9580
9582
9583
9584
9585
9586
9587
9588
9589
